In [1]:
import pandas as pd
import geopandas as gpd

In [13]:
import requests
base_url = 'http://localhost:8989/isochrone'

point_kyobashi = (34.69736984213814, 135.53042578386587)  # 京橋駅：大阪メトロ改札前
point_morinomiya = (34.68570708738395, 135.53702793425387)  # 森ノ宮駅：e METRO MOBILITY TOWN
point_bentencho = (34.668846296738934, 135.46164952833502)  # 弁天町駅：弁天町バスターミナル
point_kitatatsumi = (34.653594660888665, 135.5554439870788)  # 北巽駅：北巽バスターミナル

df_kyobashi = pd.DataFrame({
    'name': ['徒歩10分圏', '徒歩15分圏', '徒歩20分圏'],
    'time_limit': ['600', '900', '1200']
})

df_morinomiya = pd.DataFrame({
    'name': ['徒歩10分圏', '徒歩15分圏', '徒歩20分圏', '徒歩30分圏'],
    'time_limit': ['600', '900', '1200', '1800']
})
             
df_bentencho = pd.DataFrame({
    'name': ['徒歩10分圏', '徒歩15分圏', '徒歩20分圏', '徒歩30分圏'],
    'time_limit': ['600', '900', '1200', '1800']
})

df_kitatatsumi = pd.DataFrame({
    'name': ['徒歩10分圏'],
    'time_limit': ['600']
})

def get_isochrone(df, point):
    gdf_list = []
    for i, row in df.iterrows():
        query = {
            "point": f"{point[0]},{point[1]}",
            "time_limit": row['time_limit'],
            "profile": "foot",
            "reverse_flow": "false"
        }
        response = requests.get(base_url, query)
        geojson = response.json()
        gdf = gpd.GeoDataFrame.from_features(geojson['polygons'])
        gdf['name'] = row['name']
        gdf['time_limit'] = int(row['time_limit'])
        gdf_list.append(gdf)
    gdf = pd.concat(gdf_list, ignore_index=True)
    gdf = gdf.iloc[::-1].reset_index(drop=True)
    return gdf

In [14]:
gdf_kyobashi = get_isochrone(df_kyobashi, point_kyobashi)
gdf_morinomiya = get_isochrone(df_morinomiya, point_morinomiya)
gdf_bentencho = get_isochrone(df_bentencho, point_bentencho)
gdf_kitatatsumi = get_isochrone(df_kitatatsumi, point_kitatatsumi)

In [15]:
gdf_kyobashi.to_file("isochrone_kyobashi.geojson", driver='GeoJSON', encoding='utf-8')
gdf_morinomiya.to_file("isochrone_morinomiya.geojson", driver='GeoJSON', encoding='utf-8')
gdf_bentencho.to_file("isochrone_bentencho.geojson", driver='GeoJSON', encoding='utf-8')
gdf_kitatatsumi.to_file("isochrone_kitatatsumi.geojson", driver='GeoJSON', encoding='utf-8')

/Users/nagampere/Projects/graphhopper-kanto/.venv/lib/python3.12/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/Users/nagampere/Projects/graphhopper-kanto/.venv/lib/python3.12/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/Users/nagampere/Projects/graphhopper-kanto/.venv/lib/python3.12/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/Users/nagampere/Projects/graphhopper-kanto/.venv/lib/python3.12/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in 

In [16]:
import geopandas as gpd
from shapely.geometry import Point

points = [
    {'name': '京橋駅', 'geometry': Point(point_kyobashi[1], point_kyobashi[0])},
    {'name': '森ノ宮駅', 'geometry': Point(point_morinomiya[1], point_morinomiya[0])},
    {'name': '弁天町駅', 'geometry': Point(point_bentencho[1], point_bentencho[0])},
    {'name': '北巽駅', 'geometry': Point(point_kitatatsumi[1], point_kitatatsumi[0])}
]

gdf_points = gpd.GeoDataFrame(points, crs='EPSG:4326')
gdf_points.to_file("points.geojson", driver='GeoJSON', encoding='utf-8')